In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import random as random
import warnings
warnings.filterwarnings(action='once')
from datetime import date,datetime

## USAR Monthly Updates

The goal of this script is to create interesting data about the last month of play

### Read in data files

In [6]:
games = pd.read_csv("../test_all_games_open_2024-08-02.csv").drop(["Unnamed: 0"],axis=1)
games_w=pd.read_csv("../test_all_games_womens_2024-08-02.csv").drop(["Unnamed: 0"],axis=1)
games_t=games._append(games_w)
games_t

,tourney,Division,Round,Date,game_id,mT1P1,mT1P2,mT2P1,mT2P2,mT1_result,...,T1P1 Change,T1P2 Change,T2P1 Change,T2P2 Change,T1P1 ELO,T1P2 ELO,T2P1 ELO,T2P2 ELO,Min Games,Avg Elo
0,2021FLORIDA,PREMIER,POOL,21-05-22,1,RYAN FITZGERALD,JARRATT ROUSE,BRENDAN FERREIRA,RYAN MARINO,1.0,...,62.500000,62.500000,-62.500000,-62.500000,2000.000000,2000.000000,2000.000000,2000.000000,0.0,2000.000000
1,2021FLORIDA,PREMIER,POOL,21-05-22,2,RYAN FITZGERALD,JARRATT ROUSE,BRENDAN FERREIRA,RYAN MARINO,1.0,...,40.936295,40.936295,-40.936295,-40.936295,2062.500000,2062.500000,1937.500000,1937.500000,1.0,2000.000000
2,2021FLORIDA,PREMIER,POOL,21-05-22,3,COLEMAN EPPERSON,FREDRIC HINKLE,DAWSON MORGAN,MAX BILLINGHURST,1.0,...,62.500000,62.500000,-62.500000,-62.500000,2000.000000,2000.000000,2000.000000,2000.000000,0.0,2000.000000
3,2021FLORIDA,PREMIER,POOL,21-05-22,4,COLEMAN EPPERSON,FREDRIC HINKLE,DAWSON MORGAN,MAX BILLINGHURST,1.0,...,40.936295,40.936295,-40.936295,-40.936295,2062.500000,2062.500000,1937.500000,1937.500000,1.0,2000.000000
4,2021FLORIDA,PREMIER,POOL,21-05-22,5,DAVID GONZALES,PATRIC CHU,GARRETT HORNOK,NOAH LUSKUS,0.0,...,-62.500000,-62.500000,62.500000,62.500000,2000.000000,2000.000000,2000.000000,2000.000000,0.0,2000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,CHICAGO2024,WOMEN'S PRO 5.0+,P9: SEMI-FINAL,24-07-13,3694,ROSEMAY DIAZ,SOPHIA YU,KALIN MORGAN,SAMANTHA BARKLOW,1.0,...,105.931957,58.412024,-100.831854,-72.318319,1974.985950,2396.480092,2526.646910,2277.335746,31.0,2293.862175
3567,CHICAGO2024,WOMEN'S PRO 5.0+,FINAL,24-07-13,3695,ALI JENKI,OLIVIA JENKI,LOU WASHBURN,RONJA LORENZ,1.0,...,63.892353,70.271180,-59.226946,-74.642257,2945.377888,2905.687161,2935.970492,3032.355679,24.0,2954.847805
3568,CHICAGO2024,WOMEN'S PRO 5.0+,FINAL,24-07-13,3696,ALI JENKI,OLIVIA JENKI,LOU WASHBURN,RONJA LORENZ,1.0,...,42.613869,47.871839,-39.004280,-51.743316,3009.270241,2975.958342,2876.743546,2957.713423,25.0,2954.921388
3569,CHICAGO2024,WOMEN'S PRO 5.0+,P3: FINAL,24-07-13,3697,SARAH ALLEN,KARAH HUI,SEBASTIANI KIM,ELLE LOWE,0.0,...,-65.267104,-89.864716,73.967934,83.080121,2689.489534,2863.591461,2674.172453,2610.366141,135.0,2709.404897


In [10]:
players=pd.read_csv("../test_final_players_open_2024-08-02.csv").drop(["Unnamed: 0"],axis=1)
players

,name,elo,Games_Played,Tournaments_Played,days_since_played,Highest_Division,Date,Tournament
0,COLEMAN EPPERSON,2251.251855,12,1,0,Premier,2021-05-22,2021FLORIDA
1,FREDRIC HINKLE,2251.251855,12,1,0,Premier,2021-05-22,2021FLORIDA
2,NOAH LUSKUS,2192.505224,13,1,0,Premier,2021-05-22,2021FLORIDA
3,GARRETT HORNOK,2192.505224,13,1,0,Premier,2021-05-22,2021FLORIDA
4,RYAN FITZGERALD,2127.307963,12,1,0,Premier,2021-05-22,2021FLORIDA
...,...,...,...,...,...,...,...,...
154995,KYLE ROPER,483.303986,10,1,140,Contender,2024-07-13,CHICAGO2024
154996,WILL GENUARIO,483.303986,10,1,140,Contender,2024-07-13,CHICAGO2024
154997,JACOB SEGEDY,467.730180,10,1,49,Contender,2024-07-13,CHICAGO2024
154998,JD WOODRUFF,467.730180,10,1,49,Contender,2024-07-13,CHICAGO2024


### Filter for time period of interest

In [14]:
date_format = '%Y-%m-%d'
start_date=datetime.strptime('2024-07-04', date_format)
end_date=datetime.strptime('2024-08-01', date_format)
players["Date"]=[datetime.strptime(x, date_format)for x in players["Date"]]
date_format = '%y-%m-%d'
games["Date"]=[datetime.strptime(x, date_format)for x in games["Date"]]
games_w["Date"]=[datetime.strptime(x, date_format)for x in games_w["Date"]]
games_t["Date"]=[datetime.strptime(x, date_format)for x in games_t["Date"]]

TypeError: strptime() argument 1 must be str, not Timestamp

In [16]:
may_play=players[(players['Date'] >= start_date) & (players['Date'] < end_date)]
may_play

,name,elo,Games_Played,Tournaments_Played,days_since_played,Highest_Division,Date,Tournament
152498,GABE FINOCCHI,2036.216171,504,35,0,Pro,2024-07-13,CHICAGO2024
152499,THOMAS HAMILTON,1977.822384,359,27,0,Pro,2024-07-13,CHICAGO2024
152500,MAX MODEL,1902.514775,341,26,35,Pro,2024-07-13,CHICAGO2024
152501,VINCENT MATHIEU,1885.196165,181,13,0,Pro,2024-07-13,CHICAGO2024
152502,MAXIME PRINCE,1868.602788,77,6,0,Pro,2024-07-13,CHICAGO2024
...,...,...,...,...,...,...,...,...
154995,KYLE ROPER,483.303986,10,1,140,Contender,2024-07-13,CHICAGO2024
154996,WILL GENUARIO,483.303986,10,1,140,Contender,2024-07-13,CHICAGO2024
154997,JACOB SEGEDY,467.730180,10,1,49,Contender,2024-07-13,CHICAGO2024
154998,JD WOODRUFF,467.730180,10,1,49,Contender,2024-07-13,CHICAGO2024


In [18]:
may_games=games[(games['Date'] >= start_date) & (games['Date'] < end_date)]
may_games

,tourney,Division,Round,Date,game_id,mT1P1,mT1P2,mT2P1,mT2P2,mT1_result,...,T1P1 Change,T1P2 Change,T2P1 Change,T2P2 Change,T1P1 ELO,T1P2 ELO,T2P1 ELO,T2P2 ELO,Min Games,Avg Elo
24485,CHICAGO2024,OPEN PRO 5.0+,POOL,2024-07-13,26362,PAQ CLIFFORD,KIERAN ROSE,WEST PETERSON,OWEN LIPPMAN,1.0,...,13.303470,12.302865,-14.497387,-11.232496,1847.695577,1865.952397,1701.235749,1641.627366,227.0,1764.127772
24486,CHICAGO2024,OPEN PRO 5.0+,POOL,2024-07-13,26363,DAWSON MORGAN,CALEB CUMMINGS,JÃ GAGNON,EMERSON DEAN,0.0,...,-33.760042,-27.827208,29.117719,32.584835,1794.188157,1706.520632,1692.602630,1641.519222,43.0,1708.707660
24487,CHICAGO2024,OPEN PRO 5.0+,POOL,2024-07-13,26364,ETIENNE COTE,BLAKE BOSAK,JACK BUSHART,CLIFFORD LEVEQUE,1.0,...,3.883671,5.513798,-5.129737,-4.183137,1906.863364,1839.728248,1496.550411,1457.486492,89.0,1675.157129
24488,CHICAGO2024,OPEN PRO 5.0+,POOL,2024-07-13,26365,CONNOR NELSON,JOSH FRAGIACOMO,RYAN GROSS,JOE BONDI,0.0,...,-31.146642,-30.503029,31.085416,30.564962,1756.080615,1746.621933,1665.047323,1672.695892,295.0,1710.111441
24489,CHICAGO2024,OPEN PRO 5.0+,POOL,2024-07-13,26366,JUSTIN BARR,WILL PICONE,NICK WIDEMAN,JUSTIN SHAYTAR,1.0,...,10.286550,5.287634,-8.809538,-6.263959,1796.679361,1932.878719,1596.841627,1527.182807,140.0,1713.395628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25213,CHICAGO2024,CONTENDER 4.5+,SEMI-FINAL,2024-07-13,27090,TOMMY RICHARD,JUSTIN GOLDMAN,NICK GENKINGER,DYLAN NOKER,1.0,...,20.496532,22.687870,-23.183862,-20.015120,1106.557086,1075.504751,1065.746741,1020.813273,19.0,1067.155463
25214,CHICAGO2024,CONTENDER 4.5+,SEMI-FINAL,2024-07-13,27091,TOMMY RICHARD,JUSTIN GOLDMAN,NICK GENKINGER,DYLAN NOKER,1.0,...,17.642029,19.581818,-20.025997,-17.220348,1127.053618,1098.192620,1042.562879,1000.798153,20.0,1067.151817
25215,CHICAGO2024,CONTENDER 4.5+,P5: SEMI-FINAL,2024-07-13,27092,OSCAR LOZANO,YOURI BEAUREGARD-FONTAINE,ISAIAH BERGSMA,NELSON BERGSMA,1.0,...,31.685650,31.685650,-31.685650,-31.685650,956.163590,956.163590,1051.392035,1051.392035,12.0,1003.777812
25216,CHICAGO2024,CONTENDER 4.5+,P33: SEMI-FINAL,2024-07-13,27093,MICHAEL RASCATI,MAX RAIS,CHRISTOPHER MONTGOMERY,CALEB MONTGOMERY,1.0,...,29.810727,29.810727,-27.751773,-31.804070,532.706444,532.706444,571.104579,629.709828,13.0,566.556824


In [35]:
may_games_w=games_w[(games_w['Date'] >= start_date) & (games_w['Date'] < end_date)]
may_games_w

,tourney,Division,Round,Date,game_id,mT1P1,mT1P2,mT2P1,mT2P2,mT1_result,...,T1P1 Change,T1P2 Change,T2P1 Change,T2P2 Change,T1P1 ELO,T1P2 ELO,T2P1 ELO,T2P2 ELO,Min Games,Avg Elo
3322,RICHMOND2024,WOMEN'S ADVANCED 4.0+,POOL,2024-06-15,3450,KATHELEEN PHAN,KATIE PIERSON,RAFAELA BASTOS,LAURENCE VALLÃE,1.0,...,50.143622,51.775587,-25.203944,-79.677935,2587.389180,2577.398295,2361.678237,2723.710529,36.0,2562.544060
3323,RICHMOND2024,WOMEN'S ADVANCED 4.0+,POOL,2024-06-15,3451,JOELLE NGUYEN,MADDY MOREHOUSE,SHACHI PATEL,SARAH ALLEN,1.0,...,44.462460,51.365201,-48.329326,-47.428228,2629.888174,2586.927718,2552.518304,2546.916585,106.0,2579.062695
3324,RICHMOND2024,WOMEN'S ADVANCED 4.0+,POOL,2024-06-15,3452,DIANNE FONSECA,SOPHIA YU,HANNA SORENSEN,ABBY NORRIS,1.0,...,1.042248,3.244368,-0.328082,-19.107442,2710.370013,2509.716574,1592.653650,2197.068051,0.0,2252.452072
3325,RICHMOND2024,WOMEN'S ADVANCED 4.0+,POOL,2024-06-15,3453,NINA MILLER,KATE B,AMANDA JAMIESON,HOLLY YU,0.0,...,-166.042510,-33.814630,74.518850,44.759263,2197.068051,1865.046679,1925.052044,2109.339816,0.0,2024.126648
3326,RICHMOND2024,WOMEN'S ADVANCED 4.0+,POOL,2024-06-15,3454,STEF DING,TIFF DING,REESE GALLAGHER,EMMA BRUNE,1.0,...,0.130792,0.101855,-0.146732,-0.090785,2505.243656,2548.727355,1369.538417,1286.050128,18.0,1927.389889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,NATIONALS2024,4.5+ WOMEN'S,FINAL,2024-06-29,3605,ALI JENKI,KALIN MORGAN,JULIA BROCHU,KARAH HUI,0.0,...,-61.880009,-54.138582,62.915258,53.100517,2934.116790,2887.265767,2877.873093,2937.326590,122.0,2909.145560
3478,NATIONALS2024,4.5+ WOMEN'S,FINAL,2024-06-29,3606,ALI JENKI,KALIN MORGAN,JULIA BROCHU,KARAH HUI,0.0,...,-42.408838,-36.578953,43.223278,35.827219,2872.236781,2833.127185,2940.788351,2990.427107,123.0,2909.144856
3479,NATIONALS2024,4.5+ WOMEN'S,P3: FINAL,2024-06-29,3607,KATHELEEN PHAN,KATIE PIERSON,JOELLE NGUYEN,LOU WASHBURN,1.0,...,50.939342,51.001516,-54.443934,-47.544537,2676.968540,2676.587954,2658.294983,2616.011647,129.0,2656.965781
3480,NATIONALS2024,4.5+ WOMEN'S,P3: FINAL,2024-06-29,3608,KATHELEEN PHAN,KATIE PIERSON,JOELLE NGUYEN,LOU WASHBURN,0.0,...,-79.738766,-79.693936,77.177710,82.157117,2727.907882,2727.589470,2603.851049,2568.467110,130.0,2656.953878


In [37]:
may_games_t=games_t[(games_t['Date'] >= start_date) & (games_t['Date'] < end_date)]
may_games_t

,tourney,Division,Round,Date,game_id,mT1P1,mT1P2,mT2P1,mT2P2,mT1_result,...,T1P1 Change,T1P2 Change,T2P1 Change,T2P2 Change,T1P1 ELO,T1P2 ELO,T2P1 ELO,T2P2 ELO,Min Games,Avg Elo
23308,SCORCH24,CONTENDER (OPEN 4.5),POOL,2024-06-08,25185,JOSEF QUISPE,JHANAJ QUISPE,CESAR ESCALANTE,KELVIN XIAO,1.0,...,10.530610,54.652765,-39.105910,-39.105910,1183.947724,998.278935,954.424977,954.424977,0.0,1022.769153
23309,SCORCH24,CONTENDER (OPEN 4.5),POOL,2024-06-08,25186,JAXON GONZALEZ,RYAN MONTGOMERY,AUDREY SUMMERS,HUDSON SULLIVAN,1.0,...,57.623133,40.677301,-75.602443,-88.643578,1022.037715,738.947957,954.424977,1035.319293,0.0,937.682486
23310,SCORCH24,CONTENDER (OPEN 4.5),POOL,2024-06-08,25187,ETHAN SULLIVAN,GAVIN SULLIVAN,CARLOS FIGUEROA,PEYTON UNDERWOOD,1.0,...,11.271446,8.873760,-25.047398,-25.047398,1168.845641,1220.829498,954.424977,954.424977,0.0,1074.631274
23311,SCORCH24,CONTENDER (OPEN 4.5),POOL,2024-06-08,25188,RYAN HART,KORBIN MOSSMAN,JOANNA GOULD,EMMA ANGELETTI,1.0,...,88.473811,88.473811,-42.717330,-25.003087,909.134899,909.134899,1216.460283,909.177807,10.0,985.976972
23312,SCORCH24,CONTENDER (OPEN 4.5),POOL,2024-06-08,25189,JOSHUA BRANDS,LIAM WERNER,ALEX FREEMAN,SAMUEL NASH,1.0,...,26.965935,36.194753,-79.577760,-79.577760,927.047025,786.984399,954.424977,954.424977,0.0,905.720345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,NATIONALS2024,4.5+ WOMEN'S,FINAL,2024-06-29,3605,ALI JENKI,KALIN MORGAN,JULIA BROCHU,KARAH HUI,0.0,...,-61.880009,-54.138582,62.915258,53.100517,2934.116790,2887.265767,2877.873093,2937.326590,122.0,2909.145560
3478,NATIONALS2024,4.5+ WOMEN'S,FINAL,2024-06-29,3606,ALI JENKI,KALIN MORGAN,JULIA BROCHU,KARAH HUI,0.0,...,-42.408838,-36.578953,43.223278,35.827219,2872.236781,2833.127185,2940.788351,2990.427107,123.0,2909.144856
3479,NATIONALS2024,4.5+ WOMEN'S,P3: FINAL,2024-06-29,3607,KATHELEEN PHAN,KATIE PIERSON,JOELLE NGUYEN,LOU WASHBURN,1.0,...,50.939342,51.001516,-54.443934,-47.544537,2676.968540,2676.587954,2658.294983,2616.011647,129.0,2656.965781
3480,NATIONALS2024,4.5+ WOMEN'S,P3: FINAL,2024-06-29,3608,KATHELEEN PHAN,KATIE PIERSON,JOELLE NGUYEN,LOU WASHBURN,0.0,...,-79.738766,-79.693936,77.177710,82.157117,2727.907882,2727.589470,2603.851049,2568.467110,130.0,2656.953878


In [39]:
may_games.groupby(['tourney']).count()["game_id"]

tourney
FOCO2024                        28
MINNEAPOLIS-CHALLENGER-2024    186
NATIONALS2024                  158
RICHMOND2024                   583
SCORCH24                       222
Name: game_id, dtype: int64

In [41]:
may_games_w.groupby(['tourney']).count()["game_id"]

tourney
MINNEAPOLIS-CHALLENGER-2024    26
NATIONALS2024                  45
RICHMOND2024                   89
Name: game_id, dtype: int64

In [45]:
def find_total_points(df):
    return(df["mT1_score"].sum()+df["mT2_score"].sum())

Open Total Points

In [47]:
find_total_points(may_games)

37565.0

Womens' Total points

In [49]:
find_total_points(may_games_w)

5196.0

Total Points

In [51]:
find_total_points(may_games_t)

42761.0

In [59]:
may_games[(may_games["mT1_score"]>25)|(may_games["mT2_score"]>24)][["tourney","Division","Round","mT1P1","mT1P2","mT2P1","mT2P2","mT1_score","mT2_score"]]


,tourney,Division,Round,mT1P1,mT1P2,mT2P1,mT2P2,mT1_score,mT2_score
23423,SCORCH24,CONTENDER (OPEN 4.5),P3: FINAL,ANDREW MURRAY,BLAKE VANDENTOP,GAVIN SULLIVAN,ETHAN SULLIVAN,28.0,30.0
23441,SCORCH24,PREMIER (5.0),POOL,COLE MADDOX,CLAYTON STUPIENSKI,GRIFFIN SISK,ALEX NEWTON,24.0,25.0
23570,RICHMOND2024,OPEN PRO 5.0+,POOL,ETIENNE COTE,BLAKE BOSAK,GABE FINOCCHI,CHRISTIAN BENNETT,24.0,25.0
23676,RICHMOND2024,PREMIER 5.0+,POOL,TIM LACAGNINA,NICK EVANKO,NOAH JOHNSON,CHASE MEYER,24.0,25.0
23712,RICHMOND2024,PREMIER 5.0+,POOL,KURT LANNETTI,SAMUEL COREY,JAVIER VIDAL,DREW RYDER,24.0,25.0
23735,RICHMOND2024,PREMIER 5.0+,POOL,MITCHELL CARTER,ELY MARCIANO,ZAK KOVATCH,LEVI ROGERS,24.0,25.0
23745,RICHMOND2024,PREMIER 5.0+,ROUND OF 64,PEYTON STACK,HARRISON BAISDEN,LUCAS HOPE,CONNOR PRELICH,26.0,24.0
23751,RICHMOND2024,PREMIER 5.0+,ROUND OF 64,MATT CATUCCIO,MICHAEL CATUCCIO,JAVIER VIDAL,DREW RYDER,27.0,25.0
23756,RICHMOND2024,PREMIER 5.0+,ROUND OF 32,ALEX NEWTON,JOSH HUTKO,GARRETT BLANCHARD,JACOB ARZAGA,29.0,27.0
23764,RICHMOND2024,PREMIER 5.0+,ROUND OF 32,JOSEF QUISPE,JOEY REYNOLDS,CALVIN SMITH,CALEB WHITE,26.0,24.0


In [61]:
may_games_w[(may_games_w["mT1_score"]>24)|(may_games_w["mT2_score"]>24)][["tourney","Division","Round","mT1P1","mT1P2","mT2P1","mT2P2","mT1_score","mT2_score"]]


,tourney,Division,Round,mT1P1,mT1P2,mT2P1,mT2P2,mT1_score,mT2_score
3433,MINNEAPOLIS-CHALLENGER-2024,WOMEN'S DIVISION,SEMI-FINAL,ALLY NEIDERMYER,RACHEL LUCZAK,RUTHIE LUNDBERG,SOPHIA SCHMIDT,26.0,24.0


In [139]:
may_games_t.columns

Index(['tourney', 'Division', 'Round', 'Date', 'game_id', 'mT1P1', 'mT1P2',
       'mT2P1', 'mT2P2', 'mT1_result', 'mT1_score', 'mT2_score', 'prob', 'Win',
       'T1P1 Change', 'T1P2 Change', 'T2P1 Change', 'T2P2 Change', 'T1P1 ELO',
       'T1P2 ELO', 'T2P1 ELO', 'T2P2 ELO', 'Min Games', 'Avg Elo'],
      dtype='object')

In [63]:
may_games_t[((may_games_t["prob"]<0.20) & (may_games_t["Win"]==True))|((may_games_t["prob"]>.80)&(may_games_t["Win"]==False))][["tourney","Division","Round","mT1P1","mT1P2","mT2P1","mT2P2","mT1_score","mT2_score",'prob', 'Win']]


,tourney,Division,Round,mT1P1,mT1P2,mT2P1,mT2P2,mT1_score,mT2_score,prob,Win
23487,SCORCH24,PREMIER (5.0),ROUND OF 16,POLK DENMARK,JACOB SUMMERS,DAVID ROWLANDS,JOSH HUTKO,16.0,21.0,0.068165,True
23497,SCORCH24,PREMIER (5.0),ROUND OF 16,TYLER FERNANDEZ,DAN ABRAMS,BEN SAVAGE,ADAM PORTER,19.0,21.0,0.179977,True
23502,SCORCH24,PREMIER (5.0),QUARTER-FINAL,CALEB CUMMINGS,BEN GOLDFARB,JAKE MOTTERSHEAD,MIKE GARRETT,19.0,21.0,0.128214,True
23538,RICHMOND2024,OPEN PRO 5.0+,POOL,ETIENNE COTE,BLAKE BOSAK,EMERSON DEAN,JÃ GAGNON,19.0,21.0,0.095895,True
23555,RICHMOND2024,OPEN PRO 5.0+,POOL,CHRISTIAN BENNETT,GABE FINOCCHI,EMERSON DEAN,JÃ GAGNON,13.0,21.0,0.186162,True
23559,RICHMOND2024,OPEN PRO 5.0+,POOL,RYAN GROSS,JOE BONDI,THOMAS HOFFMAN,KYLE FOWLER,17.0,21.0,0.145685,True
23581,RICHMOND2024,OPEN PRO 5.0+,ROUND OF 16,KIERAN ROSE,PAQ CLIFFORD,THOMAS HOFFMAN,KYLE FOWLER,19.0,21.0,0.185721,True
23625,RICHMOND2024,PREMIER 5.0+,POOL,DALTON GILDER,BRANDON STOKES,GAVIN SULLIVAN,ETHAN SULLIVAN,15.0,7.0,0.808988,False
23728,RICHMOND2024,PREMIER 5.0+,POOL,SAM BUNZE,JOHN BUCHANAN,NATHAN MONTGOMERY,BRIAN PERLSON,16.0,21.0,0.140633,True
23753,RICHMOND2024,PREMIER 5.0+,ROUND OF 32,ANTHONY ST-MARTIN,THOMAS DÃRY,MICHAEL CATUCCIO,MATT CATUCCIO,17.0,21.0,0.106413,True


In [62]:
may_games_t[((may_games_t["prob"]<0.3) & (may_games_t["Win"]==True))|((may_games_t["prob"]>.7)&(may_games_t["Win"]==False))&(may_games_t["Round"]!="POOL")][["tourney","Division","Round","mT1P1","mT1P2","mT2P1","mT2P2","mT1_score","mT2_score",'prob', 'Win']]


,tourney,Division,Round,mT1P1,mT1P2,mT2P1,mT2P2,mT1_score,mT2_score,prob,Win
22269,STSOSGRANDSLAM2024,5.0 PREMIER,POOL,LUKE PENNINGTON,ANDREW KOSCHE,RICKY SANTIAGO,KEVIN CHEN,22.0,24.0,0.094603,True
22279,STSOSGRANDSLAM2024,5.0 PREMIER,POOL,DREW CUMMINS,MATT ORTLIP,BENJAMIN BANUELOS,CLIFTON MILLER,19.0,21.0,0.232881,True
22282,STSOSGRANDSLAM2024,5.0 PREMIER,POOL,GARRETT ASHTON,MAX HUANG,THOMAS PENCE,JOHN CHOI,24.0,25.0,0.271657,True
22293,STSOSGRANDSLAM2024,5.0 PREMIER,POOL,FREDRIC HINKLE,JACOB PAYER,MATT ORTLIP,DREW CUMMINS,17.0,21.0,0.217540,True
22296,STSOSGRANDSLAM2024,5.0 PREMIER,POOL,SUNNY GU,LUCAS PRUETT,THOMAS PENCE,JOHN CHOI,19.0,21.0,0.023827,True
...,...,...,...,...,...,...,...,...,...,...,...
3261,RALIEGH2024,WOMENS ADVANCED 4.0+,P3: FINAL,KATHELEEN PHAN,KATIE PIERSON,RAFAELA BASTOS,JOANNA GOULD,19.0,21.0,0.213788,True
3299,IDAHOCHALLENGER2024,WOMENS,QUARTER-FINAL,BETHANY DINH,KAYLA NGAI,EVA MURRAY,ELIZA MURRAY,21.0,8.0,0.851439,False
3301,IDAHOCHALLENGER2024,WOMENS,QUARTER-FINAL,JORDAN HEMMEN,SARA BRADSHAW,JACQUELINE JOHNSON,LINDY WESTENDORF,21.0,18.0,0.897093,False
3316,IDAHOCHALLENGER2024,WOMENS,P5: FINAL,LINDY WESTENDORF,JACQUELINE JOHNSON,EVA MURRAY,ELIZA MURRAY,22.0,24.0,0.261875,True
